In [1]:
from navitia_client import Client
from datetime import datetime
import unidecode

In [23]:
def get_navitia_journey():
    NAVITIA_USER = "YOUR API KEY"

    client = Client(user=NAVITIA_USER)

    client.set_region("fr-idf")

    departure = '2.364979;48.79974'
    destination = '2.290649;48.918551'

    traveler_type = 'standard'
    commercial_mode = 'Bus'

    raw_url = f'coverage/fr-idf/journeys?from={departure}&to={destination}&traveler_type={traveler_type}&allowed_id%5B%5D=commercial_mode%3A{commercial_mode}&max_nb_journeys=3&'
    response = client.raw(url=raw_url)

    response = response.json()

    # Extract routes and schedules
    journeys = response['journeys']

    # Get best journey route
    for i, journey in enumerate(journeys):
        if journey['status'] == '':
            if commercial_mode == '':
                best_route = journey
                break
            else:
                for j in range(0, len(journey['sections'])):
                    mode = journey['sections'][j].get('display_informations', '')
                    if 'physical_mode' in mode:
                        mode = mode.get('physical_mode')
                        mode = unidecode.unidecode(mode).lower()
                        if mode == commercial_mode.lower():
                            best_route = journey
                            break

    return best_route

In [3]:
best_route = get_navitia_journey()

In [4]:
def convert_duration(seconds):
    min, sec = divmod(seconds, 60)
    hour, min = divmod(min, 60)
    return '%d:%02d:%02d' % (hour, min, sec)

In [5]:
def journey_summary(best_route, trp_mean, trp_mode):
    departure_time = best_route['departure_date_time']

    datetime_obj = datetime.strptime(departure_time, '%Y%m%dT%H%M%S')
    formatted_time_dep = datetime_obj.strftime('%H:%M:%S')

    arrival_time = best_route['arrival_date_time']

    datetime_obj = datetime.strptime(arrival_time, '%Y%m%dT%H%M%S')
    formatted_time_arr = datetime_obj.strftime('%H:%M:%S')

    durations = []
    for i, transportation in enumerate(best_route['durations']):
        if best_route['durations'][transportation] > 0:
            durations.append({transportation: convert_duration(best_route['durations'][transportation])})

    distances = []
    for i, transportation in enumerate(best_route['distances']):
        if best_route['distances'][transportation] > 0:
            distances.append({transportation: best_route['distances'][transportation]})

    start = best_route['sections'][0]['from']['address']['label']
    start_coordinates = best_route['sections'][0]['from']['address']['coord']

    dest = best_route['sections'][len(best_route['sections']) - 1]['to']['address']['label']
    dest_coordinates = best_route['sections'][len(best_route['sections']) - 1]['to']['address']['coord']
    
    total_duration = durations[0]['total']

    time_components = total_duration.split(':')
    hours = int(time_components[0])
    minutes = int(time_components[1])
    seconds = int(time_components[2])

    if hours == 0:
        if minutes == 0:
            total_duration = f"{seconds} seconds"
        else:
            total_duration = f"{minutes} minutes"
    else:
        total_duration = total_duration

    distance_string = ', '.join([f"{list(d.keys())[0].capitalize()}: {list(d.values())[0]}m" for d in distances])

    print(f"""Here's the best route journey we found for you using your current transportation mode ({trp_mean}) and your preffered transportation ({trp_mode}):
          🛎️ Departure at {formatted_time_dep} from {start} 
          🧳 Arrival in {formatted_time_arr} to {dest} 
          🕖 Total duration of the trip {total_duration} 
          🛸 {distance_string}""")


In [8]:
journey_summary(best_route, "🚶‍♂️ Walking", "🚍 Bus")

Here's the best route journey we found for you using your current transportation mode (🚶‍♂️ Walking) and your preffered transportation (🚍 Bus):
          🛎️ Departure at 05:34:33 from 100 Avenue de Paris (Villejuif) 
          🧳 Arrival in 07:35:51 to 74 Rue des Bas (Gennevilliers) 
          🕖 Total duration of the trip 2:01:13 
          🛸 Walking: 1213m, Bike: 2328m


In [9]:
def get_journey_steps(best_route, commercial_mode):
    for i, section in enumerate(best_route['sections']):
        try:
            from_label = section['from'].get('address', {}).get('label') or section['from'].get('stop_point', {}).get('label') or section['from'].get('poi', {}).get('label') or ''
            to_label = section['to'].get('address', {}).get('label') or section['to'].get('stop_point', {}).get('label') or section['to'].get('poi', {}).get('label') or ''
            via_label = section.get('vias', [{}])[0].get('name', '')

            duration = section.get('duration')
            duration = convert_duration(duration)

            time_components = duration.split(':')
            hours = int(time_components[0])
            minutes = int(time_components[1])
            seconds = int(time_components[2])

            if hours == 0:
                if minutes == 0:
                    duration = f"{seconds} seconds"
                else:
                    duration = f"{minutes} minutes"
            else:
                duration = duration

            print(f"Step {i+1}. From {from_label} to {to_label} | ⏱️ Duration: {duration} ", end='')

            transport_mode = section.get('display_informations', {}).get('commercial_mode', {}) or ''
            transport_direction = section.get('display_informations', {}).get('direction', {}) or ''
            transport_code = section.get('display_informations', {}).get('code', {}) or ''

            if via_label:
                print(f"via {via_label}")
            else:
                print()

            try:
                stop_datetime = section['stop_date_times'][0].get('departure_date_time') or ''
                stop_datetime = datetime.strptime(stop_datetime, '%Y%m%dT%H%M%S')
                stop_datetime = stop_datetime.strftime('%H:%M:%S')

                print(f"🛎️ Departure Time: {stop_datetime} | ", end='')
            except:
                pass

            if commercial_mode == "Bus":
                emoji_trp = "🚍"
            elif commercial_mode == "Metro":
                emoji_trp = "🚇"
            elif commercial_mode == "RapidTransit":
                emoji_trp = "🚈"
            elif commercial_mode == "LocalTrain":
                emoji_trp = "🚆"
            elif commercial_mode == "Tramway":
                emoji_trp = "🚊"
            else:
                emoji_trp = "🛸"

            if transport_mode and stop_datetime:
                print(f"{emoji_trp} {transport_mode} Line {transport_code} heading to {transport_direction}\n")
            else:
                print()
                
        except:
            duration = section.get('duration')

            print(f"Step {i+1}. ⏳ Wait for {int(duration / 60)} min\n")

In [10]:
get_journey_steps(best_route, "Bus")

Step 1. From 100 Avenue de Paris (Villejuif) to Avenue de Paris | ⏱️ Duration: 8 minutes 

Step 2. From Avenue de Paris to LCL Campus - Place Oscar Niemeyer | ⏱️ Duration: 2 minutes 

Step 3. From LCL Campus - Place Oscar Niemeyer to Italie - Tolbiac | ⏱️ Duration: 11 minutes 

Step 4. From Italie - Tolbiac to Avenue d'Italie | ⏱️ Duration: 1 minutes 

Step 5. From Avenue d'Italie to Italie - Tolbiac (Paris) | ⏱️ Duration: 1 minutes 

Step 6. From Italie - Tolbiac (Paris) to Porte de Saint-Cloud (Paris) | ⏱️ Duration: 33 minutes 
🛎️ Departure Time: 05:59:00 | 🚍 Bus Line 62 heading to Porte de Saint-Cloud (Paris)

Step 7. From Porte de Saint-Cloud (Paris) to Porte de Saint-Cloud (Paris) | ⏱️ Duration: 1 minutes 

Step 8. ⏳ Wait for 7 min

Step 9. From Porte de Saint-Cloud (Paris) to Gabriel Péri - Métro (Asnières-sur-Seine) | ⏱️ Duration: 49 minutes 
🛎️ Departure Time: 06:40:00 | 🚍 Bus Line 175 heading to Gabriel Péri - Métro (Asnières-sur-Seine)

Step 10. From Gabriel Péri - Métro (Asn